In [1]:
import pandas as pd

In [2]:
# Load the cleaned datasets
enrollment=pd.read_csv("cleaned dataset/cleaned_enrollment_data.csv")
biometric=pd.read_csv("cleaned dataset/cleaned_biometric_data.csv")
demographic=pd.read_csv("cleaned dataset/cleaned_demographic_data.csv")

In [3]:
# Create dim_date by concatenating date columns from the three datasets
dim_date=pd.concat([enrollment['date'],biometric['date'],demographic['date']])

In [4]:
#number or rows in dim_date
dim_date.shape

(3575975,)

In [5]:
#duplicate rows in dim_date
dim_date.duplicated().sum()

np.int64(3575788)

In [6]:
#drop duplicate rows in dim_date
dim_date=dim_date.drop_duplicates()

In [7]:
#verify the shape after dropping duplicates
dim_date.shape

(187,)

In [8]:
# Convert to DataFrame only if it's currently a Series
if isinstance(dim_date, pd.Series):
    dim_date = dim_date.to_frame(name='date')

# Convert to datetime objects correctly (handling DD-MM-YYYY)
dim_date['date'] = pd.to_datetime(dim_date['date'], format='%d-%m-%Y', errors='coerce')

# Sort chronologically and fix that messy index column
dim_date = dim_date.sort_values(by='date').reset_index(drop=True)

FEATURE ENGINEERING

In [9]:
#date,year,month,quarter
dim_date['year'] = dim_date['date'].dt.year
dim_date['month'] = dim_date['date'].dt.month
dim_date['month_name'] = dim_date['date'].dt.strftime('%B')
dim_date['quarter'] = dim_date['date'].dt.quarter

In [10]:
import holidays
# Initialize holidays
india_holidays = holidays.India(years=[2025, 2026])

# IMPORTANT: Remove any rows where 'date' is Null/NaN
dim_date = dim_date.dropna(subset=['date']).copy()

#Double-check: Ensure it's in datetime format
dim_date['date'] = pd.to_datetime(dim_date['date'])

# Apply holiday check with a safety condition
def check_holiday(x):
    if pd.isna(x):
        return False
    return x in india_holidays

def get_holiday_name(x):
    if pd.isna(x):
        return None
    return india_holidays.get(x)

dim_date['is_holiday'] = dim_date['date'].apply(check_holiday)
dim_date['holiday_name'] = dim_date['date'].apply(get_holiday_name)

In [11]:
# Add Day of Week and Weekend Flag
dim_date['day_name'] = dim_date['date'].dt.day_name()
dim_date['is_weekend'] = dim_date['date'].dt.dayofweek >= 5


In [12]:
#Add India Fiscal Year (Starts in April)
dim_date['fiscal_year'] = dim_date['date'].apply(lambda x: x.year if x.month >= 4 else x.year - 1)

In [13]:

#Add School Admission Season (May, June, July)
dim_date['is_school_season'] = dim_date['month'].isin([5, 6, 7])

In [14]:
#Logic for is_month_end
# This returns True if the date is the last day of the month
dim_date['is_month_end'] = dim_date['date'].dt.is_month_end

In [15]:
# Mapping months to standard Indian seasons for societal trend analysis
def get_india_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Summer'
    elif month in [6, 7, 8, 9]:
        return 'Monsoon'
    else:
        return 'Post-Monsoon'

dim_date['season'] = dim_date['date'].dt.month.apply(get_india_season)

In [16]:
# Final column order check
column_order = [
    'date', 'year', 'month', 'month_name', 'quarter', 
    'is_holiday', 'holiday_name', 'day_name', 'is_weekend', 
    'fiscal_year', 'is_school_season', 'is_month_end', 'season'
]

In [17]:
dim_date.head(5)

,date,year,month,month_name,quarter,is_holiday,holiday_name,day_name,is_weekend,fiscal_year,is_school_season,is_month_end,season
0,2025-03-02,2025.0,3.0,March,1.0,False,None,Sunday,True,2024,False,False,Summer
1,2025-03-09,2025.0,3.0,March,1.0,False,None,Sunday,True,2024,False,False,Summer
2,2025-03-15,2025.0,3.0,March,1.0,False,None,Saturday,True,2024,False,False,Summer
3,2025-03-20,2025.0,3.0,March,1.0,False,None,Thursday,False,2024,False,False,Summer
4,2025-03-23,2025.0,3.0,March,1.0,False,None,Sunday,True,2024,False,False,Summer


In [18]:
dim_date.to_csv("dim_date.csv", index=False)

dim_geographic

In [19]:
# Create dim_date by concatenating date columns from the three datasets
dim_geography=pd.concat([enrollment[['pincode','state','district','zone']],biometric[['pincode','state','district','zone']],demographic[['pincode','state','district','zone']]], ignore_index=True)

In [20]:
print(dim_geography.shape)
print(dim_geography.columns)
print(dim_geography.duplicated(['pincode']).sum())

(3575975, 4)
Index(['pincode', 'state', 'district', 'zone'], dtype='object')
3556161


In [21]:
dim_geography.drop_duplicates(subset=['pincode'], inplace=True)
dim_geography.shape

(19814, 4)

FEATURE ENGINEERING

In [22]:
# Define hilly states based on official classification
hilly_states = [
    'ARUNACHAL PRADESH', 'HIMACHAL PRADESH', 'JAMMU & KASHMIR', 
    'MANIPUR', 'MEGHALAYA', 'MIZORAM', 'NAGALAND', 'SIKKIM', 
    'TRIPURA', 'UTTARAKHAND'
]

# Create the column using a lambda function
dim_geography['is_hilly_state'] = dim_geography['state'].apply(
    lambda x: True if str(x).upper() in hilly_states else False
)

In [23]:
# Mapping Dictionary for Tiers (Sample - expand this based on your dataset)
tier_map = {
    'MUMBAI': 'Tier 1', 'DELHI': 'Tier 1', 'BENGALURU': 'Tier 1', 
    'CHENNAI': 'Tier 1', 'HYDERABAD': 'Tier 1', 'KOLKATA': 'Tier 1',
    'PUNE': 'Tier 2', 'JAIPUR': 'Tier 2', 'LUCKNOW': 'Tier 2',
    'PATNA': 'Tier 3', 'RANCHI': 'Tier 3'
}

# Apply mapping to the correct_district column
dim_geography['tier_classification'] = dim_geography['district'].str.upper().map(tier_map)

# Fill remaining districts as 'Tier 4-6' (representing smaller towns/villages)
dim_geography['tier_classification'] = dim_geography['tier_classification'].fillna('Tier 4-6')

In [24]:
dim_geography.columns

Index(['pincode', 'state', 'district', 'zone', 'is_hilly_state',
       'tier_classification'],
      dtype='object')

In [25]:
dim_geography[['state','district']].nunique()

state        36
district    781
dtype: int64

In [26]:
dim_geography.to_csv("dim_geography.csv", index=False)